In [1]:
import pandas as pd
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf
le = preprocessing.LabelEncoder()
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [2]:
gpt3 = "/Users/annaking/Documents/Github/riskychoiceframing_AI/Experiment 1/analysis/df_gpt3.csv"
df_gpt3 = pd.read_csv(gpt3)
df_gpt3['source'] = 'AI'
humans = "/Users/annaking/Documents/Github/riskychoiceframing_AI/Experiment 1/analysis/experimentone_results.csv"
df_hum = pd.read_csv(humans)
df_hum['source'] = 'human'
df_hum = df_hum.rename(columns={'scenario_clean':'scenario'})
df_hum['frame_cat'] = le.fit_transform(df_hum['frame']) ##0 = gain, 1 = loss
df_hum['frame_rev'] = 1 - df_hum['frame_cat'] ##1 = gain, 0 = loss
df_hum['option_selected_cat'] = le.fit_transform(df_hum['option_selected']) ##0 = Prop A, 1 = Prop B
df_hum['option_selected_rev'] = 1 - df_hum['option_selected_cat'] ##1 = Prop A, 0 = Prop B
df_combined = pd.concat([df_hum, df_gpt3],axis = 0 ,ignore_index=True)

In [3]:
df_combined.scenario = df_combined.scenario.str.replace('humans', 'human')
df_combined.scenario = df_combined.scenario.str.replace('humanss', 'human')
df_combined.scenario = df_combined.scenario.str.replace('humans', 'human')
df_combined.scenario = df_combined.scenario.str.replace('animals', 'animal')

In [4]:
df_combined.groupby('source').agg({'rating_num':['mean', 'std']})

rating_num          
             mean       std
source                     
AI       3.821429  2.109995
human    3.321782  1.639203

Descriptive Stats

In [5]:
##chi-square for option selected 
contingency_table = pd.crosstab(df_combined['source'], df_combined['option_selected'])
chi, pval, _, _ = stats.chi2_contingency(contingency_table)
chi, pval

##fix this 
human_ratings = df_combined[df_combined['source'] == 'human']['rating_num']
ai_ratings = df_combined[df_combined['source'] == 'AI']['rating_num']
t_stat, p_val = ttest_ind(human_ratings, ai_ratings)
print("t-statistic:", t_stat)
print("p-value:", p_val)

t-statistic: -3.2222477325933605
p-value: 0.001298415641651965


#### Regression 

In [6]:
#function for signficance 
def sig_effect(pvalue):
    if pvalue <= .01:
        return f"""significant at 1% level"""
    elif pvalue <= .05:
        return f"""significant at 5% level"""
    elif pvalue <= .1:
        return f"""significant at 10% level"""
    else:
        return "not significant"


In [7]:
df_combined['frame_gain'] = (df_combined['frame'] == 'gain').astype(int) # gain asreference 
df_combined['source_AI'] = pd.get_dummies(df_combined['source'], drop_first=True) #AI as reference

##assumption of equal variance 
##could affect power 
##issues with confounding 

##caveat --> why this was better strategy 

In [8]:
###Simple Model
logit_model = smf.logit('option_selected_rev ~ frame_gain + source_AI', df_combined).fit()
logit_model.summary()


Optimization terminated successfully.
         Current function value: 0.647341
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     option_selected_rev   No. Observations:                 1546
Model:                           Logit   Df Residuals:                     1543
Method:                            MLE   Df Model:                            2
Date:                 Fri, 11 Aug 2023   Pseudo R-squ.:                 0.02589
Time:                         11:25:07   Log-Likelihood:                -1000.8
converged:                        True   LL-Null:                       -1027.4
Covariance Type:             nonrobust   LLR p-value:                 2.819e-12
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7876      0.080      9.902      0.000       0.632       0.944
source_AI[T.True]     0.4871      0.167      2.920      0.003       0.160       0.814
frame_gain           -0.7117      0.107     -6.661      0.000      -0.921      -0.502
=====================================================================================
"""

In [10]:
logit_reg = smf.logit('option_selected_rev ~ frame + source + scenario', df_combined).fit()
print("frame p-vale:", logit_reg.pvalues[1])
print("\nsignficicant effect? --> ", sig_effect(logit_reg.pvalues[1]))
logit_reg.summary()

Optimization terminated successfully.
         Current function value: 0.620867
         Iterations 5
frame p-vale: 2.483177736339205e-09

signficicant effect? -->  significant at 1% level


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     option_selected_rev   No. Observations:                 1546
Model:                           Logit   Df Residuals:                     1541
Method:                            MLE   Df Model:                            4
Date:                 Fri, 11 Aug 2023   Pseudo R-squ.:                 0.06572
Time:                         11:25:17   Log-Likelihood:                -959.86
converged:                        True   LL-Null:                       -1027.4
Covariance Type:             nonrobust   LLR p-value:                 3.241e-28
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.7623      0.127     -5.986      0.000      -1.012      -0.513
frame[T.loss]          0.6553      0.110      5.963      0.000       0.440       0.871
source[T.human]        0.5842      0.201      2.904      0.004       0.190       0.978
scenario[T.forest]     1.4744      0.386      3.815      0.000       0.717       2.232
scenario[T.human]      1.1148      0.131      8.488      0.000       0.857       1.372
======================================================================================
"""

In [60]:
logit_reg = smf.logit('option_selected_rev ~ frame * source, df_combined).fit()
logit_reg.summary()

Optimization terminated successfully.
         Current function value: 0.612555
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     option_selected_rev   No. Observations:                 1546
Model:                           Logit   Df Residuals:                     1538
Method:                            MLE   Df Model:                            7
Date:                 Thu, 10 Aug 2023   Pseudo R-squ.:                 0.07823
Time:                         05:40:43   Log-Likelihood:                -947.01
converged:                        True   LL-Null:                       -1027.4
Covariance Type:             nonrobust   LLR p-value:                 2.233e-31
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -0.9500      0.137     -6.943      0.000      -1.218      -0.682
frame[T.loss]                     0.8105      0.118      6.855      0.000       0.579       1.042
source[T.human]                   1.6374   7.84e+06   2.09e-07      1.000   -1.54e+07    1.54e+07
scenario[T.animals]               0.2242   7.84e+06   2.86e-08      1.000   -1.54e+07    1.54e+07
scenario[T.forest]                1.1384   7.84e+06   1.45e-07      1.000   -1.54e+07    1.54e+07
scenario[T.human]                 1.2610      0.143      8.839      0.000       0.981       1.541
scenario[T.humans]                0.2748   7.84e+06   3.51e-08      1.000   -1.54e+07    1.54e+07
frame[T.loss]:source[T.human]    -1.3675      0.340     -4.025      0.000      -2.033      -0.702
=================================================================================================
"""

In [43]:
# Convert categorical variables into dummy/indicator variables. We'll use 'AI' as our reference group.
df_combined['frame'] = pd.get_dummies(df_combined['frame'], drop_first=True) # Assuming 'Loss' as reference
#####df_combined['scenario'] = pd.get_dummies(df_combined['scenario'], drop_first=True) # Assume one scenario, e.g., 'Animals', as reference
df_combined['source'] = pd.get_dummies(df_combined['source'], drop_first=True) # 'AI' as reference

X = df_combined[['frame', 'source']]
X = sm.add_constant(X)  # Adds a constant (intercept) to your predictor variables


y_option = df_combined['option_selected']

model_option = sm.Logit(y_option, X).fit()
print(model_option.summary())


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [35]:
y_option = df_combined['option_selected']
X = 'frame'
model_option = sm.Logit(y_option, X).fit()
print(model_option.summary())

NameError: name 'X' is not defined